In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, FloatType
import pyspark.sql.functions as F

In [2]:

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/13 14:58:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/13 14:58:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Lendo Csv

In [8]:
path_countries = '/mnt/datalake/transient/departments/countries'
df_countries = spark.read.format('csv')\
.option("header", True)\
.option("sep", ",")\
.option("quote","\'")\
.option("inferSchema",True)\
.load(path_countries)
#transient\csv\olist

In [9]:
path_regions = '/mnt/datalake/transient/departments/regions'
df_regions = spark.read.format('csv')\
.option("header", True)\
.option("sep", ",")\
.option("quote","\'")\
.option("inferSchema",True)\
.load(path_regions)

In [152]:
path_jobs = '/mnt/datalake/transient/departments/jobs'
df_jobs = spark.read.format('csv')\
.option("header", True)\
.option("sep", ",")\
.option("quote","\'")\
.option("inferSchema",True)\
.load(path_jobs)

In [153]:
path_employees = '/mnt/datalake/transient/departments/employees'
df_employees = spark.read.format('csv')\
.option("header", True)\
.option("sep", ",")\
.option("quote","\'")\
.option("inferSchema",True)\
.load(path_employees)

In [11]:
df_countries.show(2)

+----------+------------+---------+
|country_id|country_name|region_id|
+----------+------------+---------+
|        AR|   Argentina|        2|
|        AU|   Australia|        3|
+----------+------------+---------+
only showing top 2 rows



In [17]:
df_regions.show(20)

+---------+--------------------+
|region_id|         region_name|
+---------+--------------------+
|        1|              Europe|
|        2|            Americas|
|        3|                Asia|
|        4|Middle East and A...|
+---------+--------------------+



In [22]:
data = [(97,"Terra Média"),(98,"Westeros"),(98,"Esteros"),(100,"Sistema Solar")]


schema = StructType([ \
    StructField("region_id",IntegerType(),True), \
    StructField("region_name",StringType(),True)
  ])

df_region2 = spark.createDataFrame(data=data,schema=schema)

In [42]:
data = [(50,"Valfenda",91),(51,"Kings Landing",98),(51,"Terra",101)]

#country_id|country_name|region_id
schema = StructType([ \
    StructField("country_id",IntegerType(),True), \
    StructField("country_name",StringType(),True), \
    StructField("region_id",IntegerType(),True),
  ])

df_countries2 = spark.createDataFrame(data=data,schema=schema)

### **Union** ###
Podemos unir dataframes que tenham o mesmo schema, o efeito seria o mesmo de empilhar os dataframes <br>


In [24]:
df_regions3 = df_regions.union(df_region2)

In [25]:
df_regions3.show()

+---------+--------------------+
|region_id|         region_name|
+---------+--------------------+
|        1|              Europe|
|        2|            Americas|
|        3|                Asia|
|        4|Middle East and A...|
|       97|         Terra Média|
|       98|            Westeros|
|       99|             Esteros|
|      100|          Via Lactea|
+---------+--------------------+



In [43]:
df_countries3 = df_countries.union(df_countries2)

In [45]:
df_countries3.show(30)

+----------+--------------------+---------+
|country_id|        country_name|region_id|
+----------+--------------------+---------+
|        AR|           Argentina|        2|
|        AU|           Australia|        3|
|        BE|             Belgium|        1|
|        BR|              Brazil|        2|
|        CA|              Canada|        2|
|        CH|         Switzerland|        1|
|        CN|               China|        3|
|        DE|             Germany|        1|
|        DK|             Denmark|        1|
|        EG|               Egypt|        4|
|        FR|              France|        1|
|        HK|            HongKong|        3|
|        IL|              Israel|        4|
|        IN|               India|        3|
|        IT|               Italy|        1|
|        JP|               Japan|        3|
|        KW|              Kuwait|        4|
|        MX|              Mexico|        2|
|        NG|             Nigeria|        4|
|        NL|         Netherlands

### **Join** ###
Outra possibilidade muito utilizada é a **junção** ou **join** entre dataframes, a junção necessita que os dataframes envolvidos tenham um campo em comum, semelhante a relação de chaves primarias e estrageiras do SQL tradicional<br>
**Tipos de junções**;<br>
• inner - Junção padrão, só realiza a junção se a mesma chave exista em todos os dataframes envolvidos ;<br>
• left - Sempre retorna os elementos do dataframe da esquerda, os caso os elementos do dataframe da esquerda não sejam encontrados, as colunas desse dataframe ;<br>
• full - ;<br>
• anti - ;<br>

### **Inner Join** ###
Outra possibilidade muito utilizada é a **junção** ou **join** entre dataframes, a junção necessita que os dataframes envolvidos tenham um campo em comum, semelhante a relação de chaves primarias e estrageiras do SQL tradicional<br>

In [48]:
condicao = df_regions3.region_id == df_countries3.region_id
df_join = df_regions3.join(df_countries3, condicao ,'inner')

In [50]:
df_join.show(30)

+---------+--------------------+----------+--------------------+---------+
|region_id|         region_name|country_id|        country_name|region_id|
+---------+--------------------+----------+--------------------+---------+
|        1|              Europe|        BE|             Belgium|        1|
|        1|              Europe|        CH|         Switzerland|        1|
|        1|              Europe|        DE|             Germany|        1|
|        1|              Europe|        DK|             Denmark|        1|
|        1|              Europe|        FR|              France|        1|
|        1|              Europe|        IT|               Italy|        1|
|        1|              Europe|        NL|         Netherlands|        1|
|        1|              Europe|        UK|      United Kingdom|        1|
|        2|            Americas|        AR|           Argentina|        2|
|        2|            Americas|        BR|              Brazil|        2|
|        2|            Am

### **Left Join** ###
Sempre retorna os elementos do dataframe da esquerda, os caso os elementos do dataframe da direita não sejam encontrados, as colunas desse dataframe aparecem como nulas;<br>

In [51]:
condicao = df_regions3.region_id == df_countries3.region_id
df_join = df_regions3.join(df_countries3, condicao ,'left')

In [35]:
df_join.show(30)

+---------+--------------------+----------+--------------------+---------+
|region_id|         region_name|country_id|        country_name|region_id|
+---------+--------------------+----------+--------------------+---------+
|        1|              Europe|        UK|      United Kingdom|        1|
|        1|              Europe|        NL|         Netherlands|        1|
|        1|              Europe|        IT|               Italy|        1|
|        1|              Europe|        FR|              France|        1|
|        1|              Europe|        DK|             Denmark|        1|
|        1|              Europe|        DE|             Germany|        1|
|        1|              Europe|        CH|         Switzerland|        1|
|        1|              Europe|        BE|             Belgium|        1|
|        2|            Americas|        US|United States of ...|        2|
|        2|            Americas|        MX|              Mexico|        2|
|        2|            Am

### **Full Join** ###
Sempre retorna os elementos do dataframe da esquerda, os caso os elementos do dataframe da direita não sejam encontrados, as colunas desse dataframe aparecem como nulas;<br>

In [52]:
condicao = df_regions3.region_id == df_countries3.region_id
df_join = df_regions3.join(df_countries3, condicao ,'full')

In [53]:
df_join.show(30)

+---------+--------------------+----------+--------------------+---------+
|region_id|         region_name|country_id|        country_name|region_id|
+---------+--------------------+----------+--------------------+---------+
|        1|              Europe|        BE|             Belgium|        1|
|        1|              Europe|        CH|         Switzerland|        1|
|        1|              Europe|        DE|             Germany|        1|
|        1|              Europe|        DK|             Denmark|        1|
|        1|              Europe|        FR|              France|        1|
|        1|              Europe|        IT|               Italy|        1|
|        1|              Europe|        NL|         Netherlands|        1|
|        1|              Europe|        UK|      United Kingdom|        1|
|        2|            Americas|        AR|           Argentina|        2|
|        2|            Americas|        BR|              Brazil|        2|
|        2|            Am

### **Anti Join** ###
Sempre retorna os elementos do dataframe da direita, quem não sejam encontrados no dataframe da esquerda;<br>

In [54]:
condicao = df_regions3.region_id == df_countries.region_id
df_join = df_regions3.join(df_countries, condicao ,'anti')

In [55]:
df_join.show()

+---------+-----------+
|region_id|region_name|
+---------+-----------+
|       97|Terra Média|
|       98|   Westeros|
|       99|    Esteros|
|      100| Via Lactea|
+---------+-----------+



In [56]:
###Pivot

In [146]:
data = [(1,"Profit",100.0),
        (2,"Profit",100.0),
        (3,"Profit",100.0),
        (4,"Profit",100.0),
        (5,"Profit",100.0),
        (6,"Profit",100.0),
        (7,"Profit",100.0),
        (8,"Profit",100.0),
        (9,"Profit",100.0),
       (10,"Profit",100.0),
       (11,"Profit",100.0),
       (12,"Profit",100.0),
         (1,"Revenue",500.0),
        (2,"Revenue",500.0),
        (3,"Revenue",500.0),
        (4,"Revenue",500.0),
        (5,"Revenue",555.0),
        (6,"Revenue",777.0),
        (7,"Revenue",800.0),
        (8,"Revenue",900.0),
        (9,"Revenue",1000.0),
       (10,"Revenue",300.0),
       (12,"Revenue",400.0)
      ]

#country_id|country_name|region_id
schema = StructType([ \
    StructField("Month",IntegerType(),True), \
    StructField("Indicator",StringType(),True), \
    StructField("Amount",FloatType(),True),
  ])

df_profit = spark.createDataFrame(data=data,schema=schema)

In [147]:
df_profit.show()

+-----+---------+------+
|Month|Indicator|Amount|
+-----+---------+------+
|    1|   Profit| 100.0|
|    2|   Profit| 100.0|
|    3|   Profit| 100.0|
|    4|   Profit| 100.0|
|    5|   Profit| 100.0|
|    6|   Profit| 100.0|
|    7|   Profit| 100.0|
|    8|   Profit| 100.0|
|    9|   Profit| 100.0|
|   10|   Profit| 100.0|
|   11|   Profit| 100.0|
|   12|   Profit| 100.0|
|    1|  Revenue| 500.0|
|    2|  Revenue| 500.0|
|    3|  Revenue| 500.0|
|    4|  Revenue| 500.0|
|    5|  Revenue| 555.0|
|    6|  Revenue| 777.0|
|    7|  Revenue| 800.0|
|    8|  Revenue| 900.0|
+-----+---------+------+
only showing top 20 rows



In [148]:
df_pivot = df_profit.groupBy("Indicator").pivot("Month").sum("Amount")
df_pivot.printSchema()
df_pivot.show(truncate=False)

root
 |-- Indicator: string (nullable = true)
 |-- 1: double (nullable = true)
 |-- 2: double (nullable = true)
 |-- 3: double (nullable = true)
 |-- 4: double (nullable = true)
 |-- 5: double (nullable = true)
 |-- 6: double (nullable = true)
 |-- 7: double (nullable = true)
 |-- 8: double (nullable = true)
 |-- 9: double (nullable = true)
 |-- 10: double (nullable = true)
 |-- 11: double (nullable = true)
 |-- 12: double (nullable = true)

+---------+-----+-----+-----+-----+-----+-----+-----+-----+------+-----+-----+-----+
|Indicator|1    |2    |3    |4    |5    |6    |7    |8    |9     |10   |11   |12   |
+---------+-----+-----+-----+-----+-----+-----+-----+-----+------+-----+-----+-----+
|Profit   |100.0|100.0|100.0|100.0|100.0|100.0|100.0|100.0|100.0 |100.0|100.0|100.0|
|Revenue  |500.0|500.0|500.0|500.0|555.0|777.0|800.0|900.0|1000.0|300.0|null |400.0|
+---------+-----+-----+-----+-----+-----+-----+-----+-----+------+-----+-----+-----+



In [149]:
months = [1,2,3,4,5,6,7,8,9,10,11,12]
df_pivot = df_profit.groupBy("Indicator").pivot("Month", months).sum("Amount")
df_pivot.printSchema()
df_pivot.show(truncate=False)

root
 |-- Indicator: string (nullable = true)
 |-- 1: double (nullable = true)
 |-- 2: double (nullable = true)
 |-- 3: double (nullable = true)
 |-- 4: double (nullable = true)
 |-- 5: double (nullable = true)
 |-- 6: double (nullable = true)
 |-- 7: double (nullable = true)
 |-- 8: double (nullable = true)
 |-- 9: double (nullable = true)
 |-- 10: double (nullable = true)
 |-- 11: double (nullable = true)
 |-- 12: double (nullable = true)

+---------+-----+-----+-----+-----+-----+-----+-----+-----+------+-----+-----+-----+
|Indicator|1    |2    |3    |4    |5    |6    |7    |8    |9     |10   |11   |12   |
+---------+-----+-----+-----+-----+-----+-----+-----+-----+------+-----+-----+-----+
|Profit   |100.0|100.0|100.0|100.0|100.0|100.0|100.0|100.0|100.0 |100.0|100.0|100.0|
|Revenue  |500.0|500.0|500.0|500.0|555.0|777.0|800.0|900.0|1000.0|300.0|null |400.0|
+---------+-----+-----+-----+-----+-----+-----+-----+-----+------+-----+-----+-----+



In [ ]:
###UnPivot

In [150]:
from pyspark.sql.functions import expr
unPivotDF = df_pivot.unpivot(['Indicator'], ['1','2','3','4','5','6','7','8','9','10','11','12'],\
                             'Month', 'Amount')

unPivotDF.show(truncate=False)


+---------+-----+------+
|Indicator|Month|Amount|
+---------+-----+------+
|Profit   |1    |100.0 |
|Profit   |2    |100.0 |
|Profit   |3    |100.0 |
|Profit   |4    |100.0 |
|Profit   |5    |100.0 |
|Profit   |6    |100.0 |
|Profit   |7    |100.0 |
|Profit   |8    |100.0 |
|Profit   |9    |100.0 |
|Profit   |10   |100.0 |
|Profit   |11   |100.0 |
|Profit   |12   |100.0 |
|Revenue  |1    |500.0 |
|Revenue  |2    |500.0 |
|Revenue  |3    |500.0 |
|Revenue  |4    |500.0 |
|Revenue  |5    |555.0 |
|Revenue  |6    |777.0 |
|Revenue  |7    |800.0 |
|Revenue  |8    |900.0 |
+---------+-----+------+
only showing top 20 rows



In [151]:
###Agregações

count()	Use groupBy() count() to return the number of rows for each group.
mean()	Returns the mean of values for each group.
max()	Returns the maximum of values for each group.
min()	Returns the minimum of values for each group.
sum()	Returns the total for values for each group.
avg()	Returns the average for values for each group.
agg()	Using groupBy() agg() function, we can calculate more than one aggregate at a time.

In [202]:
unPivotDF.count()

24

In [203]:
unPivotDF.groupBy('Indicator').mean('Amount').show()

+---------+-----------+
|Indicator|avg(Amount)|
+---------+-----------+
|   Profit|      100.0|
|  Revenue|      612.0|
+---------+-----------+



In [206]:
unPivotDF.groupBy('Indicator').sum('Amount').show()

+---------+-----------+
|Indicator|sum(Amount)|
+---------+-----------+
|   Profit|     1200.0|
|  Revenue|     6732.0|
+---------+-----------+



In [212]:
unPivotDF.groupBy('Indicator').max('Amount').show()

+---------+-----------+
|Indicator|max(Amount)|
+---------+-----------+
|   Profit|      100.0|
|  Revenue|     1000.0|
+---------+-----------+



In [213]:
unPivotDF.groupBy('Indicator').min('Amount').show()

+---------+-----------+
|Indicator|min(Amount)|
+---------+-----------+
|   Profit|      100.0|
|  Revenue|      300.0|
+---------+-----------+



In [217]:
df_grouped = unPivotDF.groupBy('Indicator')

In [218]:
from pyspark.sql import types as T, functions as F

In [224]:
df_grouped.agg(F.collect_list(F.col('Amount'))
              ).show(truncate = False)

+---------+------------------------------------------------------------------------------------+
|Indicator|collect_list(Amount)                                                                |
+---------+------------------------------------------------------------------------------------+
|Profit   |[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]|
|Revenue  |[500.0, 500.0, 500.0, 500.0, 555.0, 777.0, 800.0, 900.0, 1000.0, 300.0, 400.0]      |
+---------+------------------------------------------------------------------------------------+



In [228]:
df_jobs.show()

+------+--------------------+----------+----------+
|job_id|           jod_title|min_salary|max_salary|
+------+--------------------+----------+----------+
|     1|   Public Accountant|    4200.0|    9000.0|
|     2|  Accounting Manager|    8200.0|   16000.0|
|     3|Administration As...|    3000.0|    6000.0|
|     4|           President|   20000.0|   40000.0|
|     5|Administration Vi...|   15000.0|   30000.0|
|     6|          Accountant|    4200.0|    9000.0|
|     7|     Finance Manager|    8200.0|   16000.0|
|     8|Human Resources R...|    4000.0|    9000.0|
|     9|          Programmer|    4000.0|   10000.0|
|    10|   Marketing Manager|    9000.0|   15000.0|
|    11|Marketing Represe...|    4000.0|    9000.0|
|    12|Public Relations ...|    4500.0|   10500.0|
|    13|    Purchasing Clerk|    2500.0|    5500.0|
|    14|  Purchasing Manager|    8000.0|   15000.0|
|    15|       Sales Manager|   10000.0|   20000.0|
|    16|Sales Representative|    6000.0|   12000.0|
|    17|    

In [231]:
condicao = df_employees.job_id == df_jobs.job_id
df_jobs_joined = df_jobs.join(df_employees, condicao , 'inner').select(df_employees['*'],\
                                                                       df_jobs['jod_title'],\
                                                                       df_jobs['min_salary'],\
                                                                       df_jobs['max_salary'])

In [232]:
df_jobs_joined.show()

+-----------+-----------+----------+--------------------+------------+----------+------+-------+----------+-------------+--------------------+----------+----------+
|employee_id| first_name| last_name|               email|phone_number| hire_date|job_id| salary|manager_id|department_id|           jod_title|min_salary|max_salary|
+-----------+-----------+----------+--------------------+------------+----------+------+-------+----------+-------------+--------------------+----------+----------+
|        100|     Steven|      King|steven.king@sqltu...|515.123.4567|1987-06-17|     4|24000.0|      NULL|            9|           President|   20000.0|   40000.0|
|        101|      Neena|   Kochhar|neena.kochhar@sql...|515.123.4568|1989-09-21|     5|17000.0|       100|            9|Administration Vi...|   15000.0|   30000.0|
|        102|        Lex|   De Haan|lex.de haan@sqltu...|515.123.4569|1993-01-13|     5|17000.0|       100|            9|Administration Vi...|   15000.0|   30000.0|
|        1

In [211]:
from pyspark.sql.functions import sum,avg,max,count,mean

unPivotDF.groupBy('Indicator').agg(
    sum('Amount').alias('sum'),
    mean('Amount').alias('mean')
).show()

+---------+------+-----+
|Indicator|   sum| mean|
+---------+------+-----+
|   Profit|1200.0|100.0|
|  Revenue|6732.0|612.0|
+---------+------+-----+



In [ ]:
path_orders = '../../datalake/bronze/csv/olist/orders'
df_orders.write.format('parquet').mode('overwrite').save(path_orders )

### Selecionando e manipulando os dados

In [ ]:
df_orders.select('order_id','customer_id','order_status','order_purchase_timestamp','order_approved_at','order_delivered_carrier_date',\
                'order_delivered_customer_date','order_estimated_delivery_date').show(5,truncate=False)

In [105]:
df_orders.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: string (nullable = true)
 |-- order_approved_at: string (nullable = true)
 |-- order_delivered_carrier_date: string (nullable = true)
 |-- order_delivered_customer_date: string (nullable = true)
 |-- order_estimated_delivery_date: string (nullable = true)



In [106]:
df_orders.select('order_id','customer_id','order_status','order_purchase_timestamp','order_approved_at','order_delivered_carrier_date',\
                'order_delivered_customer_date','order_estimated_delivery_date').show(5,truncate=False)

+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|order_id                        |customer_id                     |order_status|order_purchase_timestamp|order_approved_at  |order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b7cc49136f2d6af7|9ef432eb6251297304e76186b10a928d|delivered   |2017-10-02 10:56:33     |2017-10-02 11:07:15|2017-10-04 19:55:00         |2017-10-10 21:25:13          |2017-10-18 00:00:00          |
|53cdb2fc8bc7dce0b6741e2150273451|b0830fb4747a6c6d20dea0b8c802d7ef|delivered   |2018-07-24 20:41:37     |2018-07-26 03:24:27|2018-07-26 14:3

### Colunas

As colunas são unidades de manipulação de dados do Spark. 
Podemos referencias colunas de algumas formas <br>
* col('nome_coluna') <br>
* dataframe['nome_coluna'] <br>
* dataframe.nome_coluna <br>

In [104]:
from pyspark.sql.functions import col, round
(
df_orders.select('order_id', 'customer_id', 'order_status', 
F.split(F.col('order_approved_at'), '-').getItem(0).alias('aproved_year_at'),
F.split(df_orders['order_approved_at'], '-').getItem(1).alias('aproved_month_at'),
F.split(df_orders.order_approved_at, '-').getItem(2).alias('aproved_month_at')).show(5)
)

+--------------------+--------------------+------------+---------------+----------------+----------------+
|            order_id|         customer_id|order_status|aproved_year_at|aproved_month_at|aproved_month_at|
+--------------------+--------------------+------------+---------------+----------------+----------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|           2017|              10|     02 11:07:15|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|           2018|              07|     26 03:24:27|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|   delivered|           2018|              08|     08 08:55:23|
|949d5b44dbf5de918...|f88197465ea7920ad...|   delivered|           2017|              11|     18 19:45:59|
|ad21c59c0840e6cb8...|8ab97904e6daea886...|   delivered|           2018|              02|     13 22:20:29|
+--------------------+--------------------+------------+---------------+----------------+----------------+
only showing top 5 rows



### Expressões

In [103]:
from pyspark.sql.functions import expr

(
    df_orders.select('order_id', 'customer_id', 'order_status', 
    F.expr('upper(order_status)'), )                    
    .show(5)
)

+--------------------+--------------------+------------+-------------------+
|            order_id|         customer_id|order_status|upper(order_status)|
+--------------------+--------------------+------------+-------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|          DELIVERED|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|          DELIVERED|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|   delivered|          DELIVERED|
|949d5b44dbf5de918...|f88197465ea7920ad...|   delivered|          DELIVERED|
|ad21c59c0840e6cb8...|8ab97904e6daea886...|   delivered|          DELIVERED|
+--------------------+--------------------+------------+-------------------+
only showing top 5 rows



In [113]:
from pyspark.sql.functions import expr
(
    df_orders.select('order_id', 'customer_id', 'order_status', 
    expr('upper(order_status)'),
    expr('substring(order_approved_at, 0,4) as year'),
    expr('substring(order_approved_at, 6,2) as month'),
    expr('substring(order_approved_at, 9,2) as day'))                    
    .show(5)
)

+--------------------+--------------------+------------+-------------------+----+-----+---+
|            order_id|         customer_id|order_status|upper(order_status)|year|month|day|
+--------------------+--------------------+------------+-------------------+----+-----+---+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|          DELIVERED|2017|   10| 02|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|          DELIVERED|2018|   07| 26|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|   delivered|          DELIVERED|2018|   08| 08|
|949d5b44dbf5de918...|f88197465ea7920ad...|   delivered|          DELIVERED|2017|   11| 18|
|ad21c59c0840e6cb8...|8ab97904e6daea886...|   delivered|          DELIVERED|2018|   02| 13|
+--------------------+--------------------+------------+-------------------+----+-----+---+
only showing top 5 rows



In [115]:

cols = ['order_id', 'order_status', 'order_estimated_delivery_date']
df_orders.select(cols).show(10)

+--------------------+------------+-----------------------------+
|            order_id|order_status|order_estimated_delivery_date|
+--------------------+------------+-----------------------------+
|e481f51cbdc54678b...|   delivered|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|   delivered|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|   delivered|          2018-09-04 00:00:00|
|949d5b44dbf5de918...|   delivered|          2017-12-15 00:00:00|
|ad21c59c0840e6cb8...|   delivered|          2018-02-26 00:00:00|
|a4591c265e18cb1dc...|   delivered|          2017-08-01 00:00:00|
|136cce7faa42fdb2c...|    invoiced|          2017-05-09 00:00:00|
|6514b8ad8028c9f2c...|   delivered|          2017-06-07 00:00:00|
|76c6e866289321a7c...|   delivered|          2017-03-06 00:00:00|
|e69bfb5eb88e0ed6a...|   delivered|          2017-08-23 00:00:00|
+--------------------+------------+-----------------------------+
only showing top 10 rows



In [116]:
cols = ['order_id', 'order_status', 'order_estimated_delivery_date']
df_orders.select('customer_id', *cols).show(10)

+--------------------+--------------------+------------+-----------------------------+
|         customer_id|            order_id|order_status|order_estimated_delivery_date|
+--------------------+--------------------+------------+-----------------------------+
|9ef432eb625129730...|e481f51cbdc54678b...|   delivered|          2017-10-18 00:00:00|
|b0830fb4747a6c6d2...|53cdb2fc8bc7dce0b...|   delivered|          2018-08-13 00:00:00|
|41ce2a54c0b03bf34...|47770eb9100c2d0c4...|   delivered|          2018-09-04 00:00:00|
|f88197465ea7920ad...|949d5b44dbf5de918...|   delivered|          2017-12-15 00:00:00|
|8ab97904e6daea886...|ad21c59c0840e6cb8...|   delivered|          2018-02-26 00:00:00|
|503740e9ca751ccdd...|a4591c265e18cb1dc...|   delivered|          2017-08-01 00:00:00|
|ed0271e0b7da060a3...|136cce7faa42fdb2c...|    invoiced|          2017-05-09 00:00:00|
|9bdf08b4b3b52b552...|6514b8ad8028c9f2c...|   delivered|          2017-06-07 00:00:00|
|f54a9f0e6b351c431...|76c6e866289321a7c...|

Observações:
* Podemos realizar operações sobre colunas selecionadas. 
* O DataFrame resultante resultante das operações vai obedeçer a order das colunas em que ele foi criado.

In [117]:
df_orders.show(2,truncate=False)

+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|order_id                        |customer_id                     |order_status|order_purchase_timestamp|order_approved_at  |order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b7cc49136f2d6af7|9ef432eb6251297304e76186b10a928d|delivered   |2017-10-02 10:56:33     |2017-10-02 11:07:15|2017-10-04 19:55:00         |2017-10-10 21:25:13          |2017-10-18 00:00:00          |
|53cdb2fc8bc7dce0b6741e2150273451|b0830fb4747a6c6d20dea0b8c802d7ef|delivered   |2018-07-24 20:41:37     |2018-07-26 03:24:27|2018-07-26 14:3

In [118]:
df_orders_selected =  (
    df_orders.select('order_id', 'customer_id', 'order_status', 
    expr('upper(order_status)'),
    expr('substring(order_approved_at, 0,4) as year'),
    expr('substring(order_approved_at, 6,2) as month'),
    expr('substring(order_approved_at, 9,2) as day'))   
)

In [125]:
df_orders_selected.selectExpr('order_id', 'customer_id','upper(order_status) as order_status','concat(year,"-",month,"-",day) as date').show(10)

+--------------------+--------------------+------------+----------+
|            order_id|         customer_id|order_status|      date|
+--------------------+--------------------+------------+----------+
|e481f51cbdc54678b...|9ef432eb625129730...|   DELIVERED|2017-10-02|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   DELIVERED|2018-07-26|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|   DELIVERED|2018-08-08|
|949d5b44dbf5de918...|f88197465ea7920ad...|   DELIVERED|2017-11-18|
|ad21c59c0840e6cb8...|8ab97904e6daea886...|   DELIVERED|2018-02-13|
|a4591c265e18cb1dc...|503740e9ca751ccdd...|   DELIVERED|2017-07-09|
|136cce7faa42fdb2c...|ed0271e0b7da060a3...|    INVOICED|2017-04-13|
|6514b8ad8028c9f2c...|9bdf08b4b3b52b552...|   DELIVERED|2017-05-16|
|76c6e866289321a7c...|f54a9f0e6b351c431...|   DELIVERED|2017-01-25|
|e69bfb5eb88e0ed6a...|31ad1d1b63eb99624...|   DELIVERED|2017-07-29|
+--------------------+--------------------+------------+----------+
only showing top 10 rows



###  Selecionando valores únicos

In [126]:
df_orders_selected.select('year').distinct().show()

+----+
|year|
+----+
|2016|
|2017|
|null|
|2018|
+----+



In [127]:
df_orders_selected.dropDuplicates(subset=['year']).show()

+--------------------+--------------------+------------+-------------------+----+-----+----+
|            order_id|         customer_id|order_status|upper(order_status)|year|month| day|
+--------------------+--------------------+------------+-------------------+----+-----+----+
|00b1cb0320190ca0d...|3532ba38a3fd24225...|    canceled|           CANCELED|null| null|null|
|d3c8851a6651eeff2...|957f8e082185574de...|  processing|         PROCESSING|2016|   10|  06|
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|          DELIVERED|2017|   10|  02|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|          DELIVERED|2018|   07|  26|
+--------------------+--------------------+------------+-------------------+----+-----+----+



### Filtrando registros e condições

Operadores lógicos disponíveis:
* e: &
* ou: |
* não: ~

As funções `filter()` e `where()` podem ser utilizadas no processo de filtragem.

In [134]:
df_orders_selected.filter(~(col('year') == 'null')).show(10)

+--------------------+--------------------+------------+-------------------+----+-----+---+
|            order_id|         customer_id|order_status|upper(order_status)|year|month|day|
+--------------------+--------------------+------------+-------------------+----+-----+---+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|          DELIVERED|2017|   10| 02|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|          DELIVERED|2018|   07| 26|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|   delivered|          DELIVERED|2018|   08| 08|
|949d5b44dbf5de918...|f88197465ea7920ad...|   delivered|          DELIVERED|2017|   11| 18|
|ad21c59c0840e6cb8...|8ab97904e6daea886...|   delivered|          DELIVERED|2018|   02| 13|
|a4591c265e18cb1dc...|503740e9ca751ccdd...|   delivered|          DELIVERED|2017|   07| 09|
|136cce7faa42fdb2c...|ed0271e0b7da060a3...|    invoiced|           INVOICED|2017|   04| 13|
|6514b8ad8028c9f2c...|9bdf08b4b3b52b552...|   delivered|          DELIVERED|2017

In [133]:
df_orders_selected.filter((col('year').isNull())).show(5)

+--------------------+--------------------+------------+-------------------+----+-----+----+
|            order_id|         customer_id|order_status|upper(order_status)|year|month| day|
+--------------------+--------------------+------------+-------------------+----+-----+----+
|00b1cb0320190ca0d...|3532ba38a3fd24225...|    canceled|           CANCELED|null| null|null|
|ed3efbd3a87bea76c...|191984a8ba4cbb214...|    canceled|           CANCELED|null| null|null|
|df8282afe61008dc2...|aa797b187b5466bc6...|    canceled|           CANCELED|null| null|null|
|8d4c637f1accf7a88...|b1dd715db389a2077...|    canceled|           CANCELED|null| null|null|
|7a9d4c7f9b0683378...|7f71ae48074c0cfec...|    canceled|           CANCELED|null| null|null|
+--------------------+--------------------+------------+-------------------+----+-----+----+
only showing top 5 rows



In [137]:
df_orders_selected.select('order_status').distinct().show()

+------------+
|order_status|
+------------+
|     shipped|
|    canceled|
|    invoiced|
|     created|
|   delivered|
| unavailable|
|  processing|
|    approved|
+------------+



In [136]:
(
    df_orders_selected.filter((col('year') == '2016') & (col('order_status') == 'invoiced'))
    .show(5)
)

+--------------------+--------------------+------------+-------------------+----+-----+---+
|            order_id|         customer_id|order_status|upper(order_status)|year|month|day|
+--------------------+--------------------+------------+-------------------+----+-----+---+
|c4e980a1d822db426...|88dc22aad9cf20898...|    invoiced|           INVOICED|2016|   10| 09|
|35b8e54d765e6b217...|1aaa5eaa9dd9bafb3...|    invoiced|           INVOICED|2016|   10| 07|
|711b9be9c346d9ecd...|81e4aed5ab4253757...|    invoiced|           INVOICED|2016|   10| 04|
|a6475bb7a50387e3c...|442d66f0d96f65609...|    invoiced|           INVOICED|2016|   10| 05|
|dd845e1cdb19f08d0...|01f7b7a4e25cda9ce...|    invoiced|           INVOICED|2016|   10| 10|
+--------------------+--------------------+------------+-------------------+----+-----+---+
only showing top 5 rows



In [142]:
(
    df_orders_selected.filter(((col('order_status') == 'unavailable') | (col('order_status') == 'canceled')) & (col('year') == '2017')).show(5)
  
)

+--------------------+--------------------+------------+-------------------+----+-----+---+
|            order_id|         customer_id|order_status|upper(order_status)|year|month|day|
+--------------------+--------------------+------------+-------------------+----+-----+---+
|8e24261a7e58791d1...|64a254d30eed42cd0...| unavailable|        UNAVAILABLE|2017|   11| 16|
|37553832a3a89c9b2...|7607cd563696c27ed...| unavailable|        UNAVAILABLE|2017|   08| 17|
|2f634e2cebf8c0283...|7353b0fb8e8d9675e...| unavailable|        UNAVAILABLE|2017|   09| 28|
|ee0db22a8e742b752...|aae50600d30bf2efe...| unavailable|        UNAVAILABLE|2017|   08| 24|
|6ad57aecbae806a7e...|d31dbd02ac052d662...| unavailable|        UNAVAILABLE|2017|   11| 30|
+--------------------+--------------------+------------+-------------------+----+-----+---+
only showing top 5 rows



In [143]:
(
    df_orders_selected.filter((col('order_status').isin('unavailable', 'canceled')) & (col('year') == '2017')).show(5)
)

+--------------------+--------------------+------------+-------------------+----+-----+---+
|            order_id|         customer_id|order_status|upper(order_status)|year|month|day|
+--------------------+--------------------+------------+-------------------+----+-----+---+
|8e24261a7e58791d1...|64a254d30eed42cd0...| unavailable|        UNAVAILABLE|2017|   11| 16|
|37553832a3a89c9b2...|7607cd563696c27ed...| unavailable|        UNAVAILABLE|2017|   08| 17|
|2f634e2cebf8c0283...|7353b0fb8e8d9675e...| unavailable|        UNAVAILABLE|2017|   09| 28|
|ee0db22a8e742b752...|aae50600d30bf2efe...| unavailable|        UNAVAILABLE|2017|   08| 24|
|6ad57aecbae806a7e...|d31dbd02ac052d662...| unavailable|        UNAVAILABLE|2017|   11| 30|
+--------------------+--------------------+------------+-------------------+----+-----+---+
only showing top 5 rows



In [144]:
(
    df_orders_selected
    .filter((col('order_status').isin('unavailable', 'canceled')))
    .filter((col('year') == '2017'))
    .show(5)
)

+--------------------+--------------------+------------+-------------------+----+-----+---+
|            order_id|         customer_id|order_status|upper(order_status)|year|month|day|
+--------------------+--------------------+------------+-------------------+----+-----+---+
|8e24261a7e58791d1...|64a254d30eed42cd0...| unavailable|        UNAVAILABLE|2017|   11| 16|
|37553832a3a89c9b2...|7607cd563696c27ed...| unavailable|        UNAVAILABLE|2017|   08| 17|
|2f634e2cebf8c0283...|7353b0fb8e8d9675e...| unavailable|        UNAVAILABLE|2017|   09| 28|
|ee0db22a8e742b752...|aae50600d30bf2efe...| unavailable|        UNAVAILABLE|2017|   08| 24|
|6ad57aecbae806a7e...|d31dbd02ac052d662...| unavailable|        UNAVAILABLE|2017|   11| 30|
+--------------------+--------------------+------------+-------------------+----+-----+---+
only showing top 5 rows



### Utilizando expressões no filtro

In [145]:
(
    df_orders_selected
    .filter('order_status in ("unavailable", "canceled") and year == "2017"')
    .show(5)
)

+--------------------+--------------------+------------+-------------------+----+-----+---+
|            order_id|         customer_id|order_status|upper(order_status)|year|month|day|
+--------------------+--------------------+------------+-------------------+----+-----+---+
|8e24261a7e58791d1...|64a254d30eed42cd0...| unavailable|        UNAVAILABLE|2017|   11| 16|
|37553832a3a89c9b2...|7607cd563696c27ed...| unavailable|        UNAVAILABLE|2017|   08| 17|
|2f634e2cebf8c0283...|7353b0fb8e8d9675e...| unavailable|        UNAVAILABLE|2017|   09| 28|
|ee0db22a8e742b752...|aae50600d30bf2efe...| unavailable|        UNAVAILABLE|2017|   08| 24|
|6ad57aecbae806a7e...|d31dbd02ac052d662...| unavailable|        UNAVAILABLE|2017|   11| 30|
+--------------------+--------------------+------------+-------------------+----+-----+---+
only showing top 5 rows



#### Observações
Quando nos referimos às colunas por meio da função `col()`, temos acesso à diversos métodos das colunas que podem ser utilizados para auxliar na filtragem do DataFrame. Alguns deles são:
* `isin()`: checa se a coluna contém os valores listados na função.
* `contains()`: utilizado para verificar se uma coluna de texto contém algum padrão especificado (não aceita regex). Aceita uma outra coluna de texto.
* `like()`: utilizado para verificar se uma coluna de texto contém algum padrão especificado (não aceita regex). Funciona de forma similar ao "LIKE" do SQL.
* `rlike()`: utilizado para verificar se uma coluna de texto contém algum padrão especificado (**aceita regex**). Funciona de forma similar ao "RLIKE" do SQL.
* `startswith()`: utilizado para verificar se uma coluna de texto começa com algum padrão especificado (**aceita regex**).
* `endswith()`: utilizado para verificar se uma coluna de texto termina com algum padrão especificado (**aceita regex**).
* `between()`: checa se os valores da coluna estão dentro do intervalo especificado. Os dois lados do intervalo são inclusivos.
* `isNull()`: retorna True se o valor da coluna é nulo
* `isNotNull()`: retorna True se o valor da coluna não é nulo

Outros métodos úteis:
* `alias()/name()`: usado para renomear as colunas em operações como select() e agg()
* `astype()/cast()`: usado para mudar o tipo das colunas. Aceita tanto um string como um tipo especificado pelo módulo pyspark.sql.types
* `substr()`: utilizado para cortar um string com base em índices dos caracteres 